In [1]:
import pywikibot
import requests
import sys
import os 
from pathlib import Path
import json


# https://stackoverflow.com/a/66303932
project_path = Path(os.path.dirname(os.path.realpath("__file__"))).parent
sys.path.append(str(project_path))
sys.path.append(str(project_path / 'scripts'))

import scripts.utils.wikidata_utils as wd


In [2]:
site = pywikibot.Site("test", "wikidata")
repo = site.data_repository()



In [3]:
new_item = pywikibot.ItemPage(repo, title="Q225211")
new_item.exists()

True

In [4]:
new_item

ItemPage('Q225211')

In [5]:

print(json.dumps(new_item.toJSON(), indent=2))

{
  "labels": {
    "en": {
      "language": "en",
      "value": "foobar"
    },
    "fr": {
      "language": "fr",
      "value": "fr foo"
    }
  },
  "descriptions": {
    "en": {
      "language": "en",
      "value": "short description text"
    }
  },
  "aliases": {
    "en": [
      {
        "language": "en",
        "value": "foobar 1"
      },
      {
        "language": "en",
        "value": "foobar 2"
      },
      {
        "language": "en",
        "value": "foobar 3"
      }
    ]
  },
  "claims": {
    "P131": [
      {
        "mainsnak": {
          "snaktype": "value",
          "property": "P131",
          "datatype": "wikibase-item",
          "datavalue": {
            "value": {
              "entity-type": "item",
              "numeric-id": 4546
            },
            "type": "wikibase-entityid"
          }
        },
        "type": "statement",
        "id": "Q225211$5347e092-8ad3-4654-9e9e-364cd26fcf5e",
        "rank": "normal",
        "qualifier

In [6]:
new_item.full_url()

'https://test.wikidata.org/wiki/Q225211'

In [7]:
item_dict = new_item.get()

In [8]:
print(json.dumps(wd.format_display_item(new_item) , indent=2))

??? ids 34
{
  "labels": {
    "en": "foobar",
    "fr": "fr foo"
  },
  "descriptions": {
    "en": "short description text"
  },
  "aliases": {
    "en": [
      "foobar 1",
      "foobar 2",
      "foobar 3"
    ]
  },
  "claims": {
    "P131": [
      {
        "main_snak": {
          "property": "P131",
          "property_value": "basic form of government",
          "data_type": "wikibase-item",
          "data_value": {
            "id": "Q4546",
            "value": "axHECMfK",
            "url": "https://test.wikidata.org/wiki/Q4546"
          }
        },
        "id": "Q225211$5347e092-8ad3-4654-9e9e-364cd26fcf5e",
        "qualifiers": {
          "P100": [
            {
              "property": "P100",
              "property_value": "top-level domainn",
              "data_type": "wikibase-item",
              "data_value": {
                "id": "Q35409",
                "value": "STNVEGFp",
                "url": "https://test.wikidata.org/wiki/Q35409"
             